In [ ]:
# CML3014N Machine Learning Assignment 2
# Data Source:
#     1. Student Performance https://archive.ics.uci.edu/dataset/320/student+performance
#     2. Wine Quality https://archive.ics.uci.edu/dataset/186/wine+quality

# Dataset Description:
#    1. Student Performance mixed dataset with both numerical and categorical data
#       G1, G2, G3 - numerical from 0 to 20 (the target variable. G3 is the final grade)
#    2. Wine Quality dataset with numerical continuous
#       quality - score between 0 and 10 (the target variable)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt

pd.set_option("display.max_columns", None)

In [2]:
# Step 1: Load the dataset
wineRed_df = pd.read_csv('data/winequality-red.csv', delimiter=';')
wineWhite_df = pd.read_csv('data/winequality-white.csv', delimiter=';')
stdMath_df = pd.read_csv('data/student-mat.csv', delimiter=';')
stdPor_df = pd.read_csv('data/student-por.csv', delimiter=';')

# Display the first few rows of the dataset
print("Wine Red Dataset")
print("Total number of columns: ", wineRed_df.shape[1], "Total number of rows: ", wineRed_df.shape[0])
display(wineRed_df.head())

print("Wine White Dataset")
print("Total number of columns: ", wineWhite_df.shape[1], "Total number of rows: ", wineWhite_df.shape[0])
display(wineWhite_df.head())

print("Student Math Dataset")
print("Total number of columns: ", stdMath_df.shape[1], "Total number of rows: ", stdMath_df.shape[0])
display(stdMath_df.head())

print("Student Portuguese Dataset")
print("Total number of columns: ", stdPor_df.shape[1], "Total number of rows: ", stdPor_df.shape[0])
display(stdPor_df.head())

Wine Red Dataset
Total number of columns:  12 Total number of rows:  1599


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Wine White Dataset
Total number of columns:  12 Total number of rows:  4898


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


Student Math Dataset
Total number of columns:  33 Total number of rows:  395


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


Student Portuguese Dataset
Total number of columns:  33 Total number of rows:  649


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [3]:
# Step 2: Preprocess the dataset
# * The model is MLP, only handle numerical data.
# * Wine data has only numerical data, each feature is in different scale, thus need to normalize the data.
# * Student data has both numerical and categorical data, need to handle categorical data with string indexer and one-hot encoding.

# Step 2.1 Preprocess the wine dataset
# Prepare both normalized and standardized dataset
scaler = MinMaxScaler()
wineRed_df_norm = pd.DataFrame(scaler.fit_transform(wineRed_df), columns=wineRed_df.columns)
wineWhite_df_norm = pd.DataFrame(scaler.fit_transform(wineWhite_df), columns=wineWhite_df.columns)

scaler = StandardScaler()
wineRed_df_std = pd.DataFrame(scaler.fit_transform(wineRed_df), columns=wineRed_df.columns)
wineWhite_df_std = pd.DataFrame(scaler.fit_transform(wineWhite_df), columns=wineWhite_df.columns)

print("Wine Red Dataset Normalized")
print("Total number of columns: ", wineRed_df_norm.shape[1], "Total number of rows: ", wineRed_df_norm.shape[0])
display(wineRed_df_norm.head())
print("Wine Red Dataset Standardized")
print("Total number of columns: ", wineRed_df_std.shape[1], "Total number of rows: ", wineRed_df_std.shape[0])
display(wineRed_df_std.head())

print("Wine White Dataset Normalized")
print("Total number of columns: ", wineWhite_df_norm.shape[1], "Total number of rows: ", wineWhite_df_norm.shape[0])
display(wineWhite_df_norm.head())
print("Wine White Dataset Standardized")
print("Total number of columns: ", wineWhite_df_std.shape[1], "Total number of rows: ", wineWhite_df_std.shape[0])
display(wineWhite_df_std.head())

# Step 2.2 Preprocess the student dataset
# Handle categorical data using label encoder
# Column to process School, sex, address, famsize, pstatus, medu, fedu, mjob, fjob, reason, guardian, schoolsup, famsup, paid, activities, nursery, higher, internet, romantic

# First encode the categorical data
columns = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
numerical_columns = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']
# Label encoding for categorical data
labelEncoder = LabelEncoder()
stdMath_df_le = stdMath_df.copy()
stdPor_df_le = stdPor_df.copy()
for column in columns:
    stdMath_df_le[column] = labelEncoder.fit_transform(stdMath_df[column])
    stdPor_df_le[column] = labelEncoder.fit_transform(stdPor_df[column])

oneHotEncoder = OneHotEncoder(sparse_output=False)
stdMath_df_ohe = pd.DataFrame(oneHotEncoder.fit_transform(stdMath_df[columns]), columns=oneHotEncoder.get_feature_names_out(columns))
stdPor_df_ohe = pd.DataFrame(oneHotEncoder.fit_transform(stdPor_df[columns]), columns=oneHotEncoder.get_feature_names_out(columns))
stdMath_df_ohe = pd.concat([stdMath_df_ohe, stdMath_df[numerical_columns]], axis=1)
stdPor_df_ohe = pd.concat([stdPor_df_ohe, stdPor_df[numerical_columns]], axis=1)

# Second normalize and standardize the dataset
scaler = MinMaxScaler()
stdMath_df_le_norm = pd.DataFrame(scaler.fit_transform(stdMath_df_le), columns=stdMath_df_le.columns)
stdMath_df_ohe_norm = pd.DataFrame(scaler.fit_transform(stdMath_df_ohe), columns=stdMath_df_ohe.columns)
stdPor_df_le_norm = pd.DataFrame(scaler.fit_transform(stdPor_df_le), columns=stdPor_df_le.columns)
stdPor_df_ohe_norm = pd.DataFrame(scaler.fit_transform(stdPor_df_ohe), columns=stdPor_df_ohe.columns)

scaler = StandardScaler()
stdMath_df_le_std = pd.DataFrame(scaler.fit_transform(stdMath_df_le), columns=stdMath_df_le.columns)
stdMath_df_ohe_std = pd.DataFrame(scaler.fit_transform(stdMath_df_ohe), columns=stdMath_df_ohe.columns)
stdPor_df_le_std = pd.DataFrame(scaler.fit_transform(stdPor_df_le), columns=stdPor_df_le.columns)
stdPor_df_ohe_std = pd.DataFrame(scaler.fit_transform(stdPor_df_ohe), columns=stdPor_df_ohe.columns)

# For student Math dataset
print("Student Math Dataset Label Encoded")
print("Total number of columns: ", stdMath_df_le.shape[1], "Total number of rows: ", stdMath_df_le.shape[0])
display(stdMath_df_le.head())
print("Student Math Dataset Label Encoded Normalized")
print("Total number of columns: ", stdMath_df_le_norm.shape[1], "Total number of rows: ", stdMath_df_le_norm.shape[0])
display(stdMath_df_le_norm.head())
print("Student Math Dataset Label Encoded Standardized")
print("Total number of columns: ", stdMath_df_le_std.shape[1], "Total number of rows: ", stdMath_df_le_std.shape[0])
display(stdMath_df_le_std.head())

print("Student Math Dataset One Hot Encoded")
print("Total number of columns: ", stdMath_df_ohe.shape[1], "Total number of rows: ", stdMath_df_ohe.shape[0])
display(stdMath_df_ohe.head())
print("Student Math Dataset One Hot Encoded Normalized")
print("Total number of columns: ", stdMath_df_ohe_norm.shape[1], "Total number of rows: ", stdMath_df_ohe_norm.shape[0])
display(stdMath_df_ohe_norm.head())
print("Student Math Dataset One Hot Encoded Standardized")
print("Total number of columns: ", stdMath_df_ohe_std.shape[1], "Total number of rows: ", stdMath_df_ohe_std.shape[0])
display(stdMath_df_ohe_std.head())

# For student Portuguese dataset
print("Student Portuguese Dataset Label Encoded")
print("Total number of columns: ", stdPor_df_le.shape[1], "Total number of rows: ", stdPor_df_le.shape[0])
display(stdPor_df_le.head())
print("Student Portuguese Dataset Label Encoded Normalized")
print("Total number of columns: ", stdPor_df_le_norm.shape[1], "Total number of rows: ", stdPor_df_le_norm.shape[0])
display(stdPor_df_le_norm.head())
print("Student Portuguese Dataset Label Encoded Standardized")
print("Total number of columns: ", stdPor_df_le_std.shape[1], "Total number of rows: ", stdPor_df_le_std.shape[0])
display(stdPor_df_le_std.head())

print("Student Portuguese Dataset One Hot Encoded")
print("Total number of columns: ", stdPor_df_ohe.shape[1], "Total number of rows: ", stdPor_df_ohe.shape[0])
display(stdPor_df_ohe.head())
print("Student Portuguese Dataset One Hot Encoded Normalized")
print("Total number of columns: ", stdPor_df_ohe_norm.shape[1], "Total number of rows: ", stdPor_df_ohe_norm.shape[0])
display(stdPor_df_ohe_norm.head())
print("Student Portuguese Dataset One Hot Encoded Standardized")
print("Total number of columns: ", stdPor_df_ohe_std.shape[1], "Total number of rows: ", stdPor_df_ohe_std.shape[0])
display(stdPor_df_ohe_std.head())

Wine Red Dataset Normalized
Total number of columns:  12 Total number of rows:  1599


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4
1,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385,0.4
2,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385,0.4
3,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385,0.6
4,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4


Wine Red Dataset Standardized
Total number of columns:  12 Total number of rows:  1599


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,-0.528360,0.961877,-1.391472,-0.453218,-0.243707,-0.466193,-0.379133,0.558274,1.288643,-0.579207,-0.960246,-0.787823
1,-0.298547,1.967442,-1.391472,0.043416,0.223875,0.872638,0.624363,0.028261,-0.719933,0.128950,-0.584777,-0.787823
2,-0.298547,1.297065,-1.186070,-0.169427,0.096353,-0.083669,0.229047,0.134264,-0.331177,-0.048089,-0.584777,-0.787823
3,1.654856,-1.384443,1.484154,-0.453218,-0.264960,0.107592,0.411500,0.664277,-0.979104,-0.461180,-0.584777,0.450848
4,-0.528360,0.961877,-1.391472,-0.453218,-0.243707,-0.466193,-0.379133,0.558274,1.288643,-0.579207,-0.960246,-0.787823


Wine White Dataset Normalized
Total number of columns:  12 Total number of rows:  4898


...

,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,paid_no,paid_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,18,4,4,2,2,0,4,3,4,1,1,3,4,0,11,11
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,17,1,1,1,2,0,5,3,3,1,1,3,2,9,11,11
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,15,1,1,1,2,0,4,3,2,2,3,3,6,12,13,12
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,15,4,2,1,3,0,3,2,2,1,1,5,0,14,14,14
4,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,16,3,3,1,2,0,4,3,2,1,2,5,0,11,13,13


Student Portuguese Dataset One Hot Encoded Normalized
Total number of columns:  59 Total number of rows:  649


,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,paid_no,paid_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.428571,1.00,1.00,0.333333,0.333333,0.0,0.75,0.50,0.75,0.00,0.00,0.5,0.1250,0.000000,0.578947,0.578947
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.285714,0.25,0.25,0.000000,0.333333,0.0,1.00,0.50,0.50,0.00,0.00,0.5,0.0625,0.473684,0.578947,0.578947
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.000000,0.25,0.25,0.000000,0.333333,0.0,0.75,0.50,0.25,0.25,0.50,0.5,0.1875,0.631579,0.684211,0.631579
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000,1.00,0.50,0.000000,0.666667,0.0,0.50,0.25,0.25,0.00,0.00,1.0,0.0000,0.736842,0.736842,0.736842
4,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.142857,0.75,0.75,0.000000,0.333333,0.0,0.75,0.50,0.25,0.00,0.25,1.0,0.0000,0.578947,0.684211,0.684211


Student Portuguese Dataset One Hot Encoded Standardized
Total number of columns:  59 Total number of rows:  649


,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,paid_no,paid_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0.730944,-0.730944,0.833377,-0.833377,-0.660182,0.660182,0.648175,-0.648175,2.666927,-2.666927,1.951258,-0.282607,-0.812309,-0.514886,-0.353247,-0.263045,-0.19168,-1.140797,-0.621894,4.126473,1.130130,-0.545894,-0.353247,-0.53161,-0.555399,0.652973,-0.259681,-2.923032,2.923032,1.259229,-1.259229,0.252853,-0.252853,0.971140,-0.971140,-0.495663,0.495663,-0.344914,0.344914,1.816043,-1.816043,0.763496,-0.763496,1.031695,1.310216,1.540715,0.576718,0.083653,-0.374305,0.072606,-0.171647,0.693785,-0.543555,-0.997695,-0.371042,0.073433,-4.155470,-0.195820,-0.280658
1,0.730944,-0.730944,0.833377,-0.833377,-0.660182,0.660182,0.648175,-0.648175,-0.374963,0.374963,1.951258,-0.282607,-0.812309,-0.514886,-0.353247,-0.263045,-0.19168,0.876580,-0.621894,-0.242338,1.130130,-0.545894,-0.353247,-0.53161,1.800508,-1.531457,-0.259681,0.342110,-0.342110,-0.794137,0.794137,0.252853,-0.252853,0.971140,-0.971140,2.017502,-2.017502,-0.344914,0.344914,-0.550648,0.550648,0.763496,-0.763496,0.210137,-1.336039,-1.188832,-0.760032,0.083653,-0.374305,1.119748,-0.171647,-0.157380,-0.543555,-0.997695,-0.371042,-0.357863,-0.874570,-0.195820,-0.280658
2,0.730944,-0.730944,0.833377,-0.833377,-0.660182,0.660182,-1.542792,1.542792,-0.374963,0.374963,1.951258,-0.282607,-0.812309,-0.514886,-0.353247,-0.263045,-0.19168,0.876580,-0.621894,-0.242338,-0.884854,-0.545894,2.830881,-0.53161,-0.555399,0.652973,-0.259681,-2.923032,2.923032,1.259229,-1.259229,0.252853,-0.252853,0.971140,-0.971140,-0.495663,0.495663,-0.344914,0.344914,-0.550648,0.550648,0.763496,-0.763496,-1.432980,-1.336039,-1.188832,-0.760032,0.083653,-0.374305,0.072606,-0.171647,-1.008546,0.538553,0.560678,-0.371042,0.504730,0.219064,0.491137,0.029116
3,0.730944,-0.730944,0.833377,-0.833377,-0.660182,0.660182,0.648175,-0.648175,-0.374963,0.374963,-0.512490,3.538479,-0.812309,-0.514886,-0.353247,-0.263045,-0.19168,-1.140797,1.607991,-0.242338,-0.884854,1.831858,-0.353247,-0.53161,-0.555399,0.652973,-0.259681,0.342110,-0.342110,-0.794137,0.794137,0.252853,-0.252853,-1.029717,1.029717,-0.495663,0.495663,-0.344914,0.344914,-0.550648,0.550648,-1.309764,1.309764,-1.432980,1.310216,-0.278983,-0.760032,1.290114,-0.374305,-0.974536,-1.123771,-1.008546,-0.543555,-0.997695,1.012903,-0.789159,0.948153,0.834615,0.648663
4,0.730944,-0.730944,0.833377,-0.833377,-0.660182,0.660182,0.648175,-0.648175,-0.374963,0.374963,-0.512490,-0.282607,1.231058,-0.514886,-0.353247,-0.263045,-0.19168,0.876580,-0.621894,-0.242338,-0.884854,1.831858,-0.353247,-0.53161,1.800508,-1.531457,-0.259681,0.342110,-0.342110,-0.794137,0.794137,0.252853,-0.252853,0.971140,-0.971140,-0.495663,0.495663,-0.344914,0.344914,1.816043,-1.816043,0.763496,-0.763496,-0.611422,0.428131,0.630866,-0.760032,0.083653,-0.374305,0.072606,-0.171647,-1.008546,-0.543555,-0.218508,1.012903,-0.789159,-0.145481,0.491137,0.338889


In [4]:
# Step 3: Split the dataset into training and testing
# * Use 80% of the data for training and 20% for testing
# * For each dataset, split into input features and target
# * For wine dataset, the target is quality
# * For student dataset, the target is G3

# Step 3.1 Split the wine dataset with train_test_split, x is features, y is target
# ! Possible Combination of feature and target for wine dataset
# ! 1. ori feature - ori target
# ! 2. norm feature - norm target
# ! 3. std feature - std target
# ! 4. norm feature - ori target
# ! 5. std feature - ori target
# Wine Red dataset
# 1. ori feature - ori target
wodeRed_x_ori = wineRed_df.drop(columns=['quality'])
wineRed_y_ori = wineRed_df['quality']  # Target variable
wineRed_x_train_ori, wineRed_x_test_ori, wineRed_y_train_ori, wineRed_y_test_ori = train_test_split(wodeRed_x_ori, wineRed_y_ori, test_size=0.2, random_state=42)
# 2. norm feature - norm target
wineRed_x_norm = wineRed_df_norm.drop(columns=['quality'])
wineRed_y_norm = wineRed_df_norm['quality']  # Target variable
wineRed_x_train_norm, wineRed_x_test_norm, wineRed_y_train_norm, wineRed_y_test_norm = train_test_split(wineRed_x_norm, wineRed_y_norm, test_size=0.2, random_state=42)
# 3. std feature - std target
wineRed_x_std = wineRed_df_std.drop(columns=['quality'])
wineRed_y_std = wineRed_df_std['quality']  # Target variable
wineRed_x_train_std, wineRed_x_test_std, wineRed_y_train_std, wineRed_y_test_std = train_test_split(wineRed_x_std, wineRed_y_std, test_size=0.2, random_state=42)
# 4. norm feature - ori target
wineRed_x_norm_ori = wineRed_df_norm.drop(columns=['quality'])
wineRed_y_norm_ori = wineRed_df['quality']  # Target variable
wineRed_x_train_norm_ori, wineRed_x_test_norm_ori, wineRed_y_train_norm_ori, wineRed_y_test_norm_ori = train_test_split(wineRed_x_norm_ori, wineRed_y_norm_ori, test_size=0.2, random_state=42)
# 5. std feature - ori target
wineRed_x_std_ori = wineRed_df_std.drop(columns=['quality'])
wineRed_y_std_ori = wineRed_df['quality']  # Target variable
wineRed_x_train_std_ori, wineRed_x_test_std_ori, wineRed_y_train_std_ori, wineRed_y_test_std_ori = train_test_split(wineRed_x_std_ori, wineRed_y_std_ori, test_size=0.2, random_state=42)

# Wine White dataset
# 1. ori feature - ori target
wineWhite_x_ori = wineWhite_df.drop(columns=['quality'])
wineWhite_y_ori = wineWhite_df['quality']  # Target variable
wineWhite_x_train_ori, wineWhite_x_test_ori, wineWhite_y_train_ori, wineWhite_y_test_ori = train_test_split(wineWhite_x_ori, wineWhite_y_ori, test_size=0.2, random_state=42)
# 2. norm feature - norm target
wineWhite_x_norm = wineWhite_df_norm.drop(columns=['quality'])
wineWhite_y_norm = wineWhite_df_norm['quality']  # Target variable
wineWhite_x_train_norm, wineWhite_x_test_norm, wineWhite_y_train_norm, wineWhite_y_test_norm = train_test_split(wineWhite_x_norm, wineWhite_y_norm, test_size=0.2, random_state=42)
# 3. std feature - std target
wineWhite_x_std = wineWhite_df_std.drop(columns=['quality'])
wineWhite_y_std = wineWhite_df_std['quality']  # Target variable
wineWhite_x_train_std, wineWhite_x_test_std, wineWhite_y_train_std, wineWhite_y_test_std = train_test_split(wineWhite_x_std, wineWhite_y_std, test_size=0.2, random_state=42)
# 4. norm feature - ori target
wineWhite_x_norm_ori = wineWhite_df_norm.drop(columns=['quality'])
wineWhite_y_norm_ori = wineWhite_df['quality']  # Target variable
wineWhite_x_train_norm_ori, wineWhite_x_test_norm_ori, wineWhite_y_train_norm_ori, wineWhite_y_test_norm_ori = train_test_split(wineWhite_x_norm_ori, wineWhite_y_norm_ori, test_size=0.2, random_state=42)
# 5. std feature - ori target
wineWhite_x_std_ori = wineWhite_df_std.drop(columns=['quality'])
wineWhite_y_std_ori = wineWhite_df['quality']  # Target variable
wineWhite_x_train_std_ori, wineWhite_x_test_std_ori, wineWhite_y_train_std_ori, wineWhite_y_test_std_ori = train_test_split(wineWhite_x_std_ori, wineWhite_y_std_ori, test_size=0.2, random_state=42)

# Step 3.2 Split the student dataset with train_test_split, x is features, y is target
# ! Possible Combination of feature and target for student dataset
# ! 1. le feature - ori target
# ! 2. le-norm feature - ori target
# ! 3. le-std feature - ori target
# ! 4. ohe feature - ori target
# ! 5. ohe-norm feature - ori target
# ! 6. ohe-std feature - ori target
# ! 7. le feature - le target
# ! 8. le-norm feature - le target
# ! 9. le-std feature - le target
# ! 10. ohe feature - ohe target
# ! 11. ohe-norm feature - ohe target
# ! 12. ohe-std feature - ohe target

# Student Math dataset
# 1. le feature - ori target
stdMath_x_le = stdMath_df_le.drop(columns=['G3'])
stdMath_y_le = stdMath_df_le['G3']  # Target variable
stdMath_x_train_le, stdMath_x_test_le, stdMath_y_train_le, stdMath_y_test_le = train_test_split(stdMath_x_le, stdMath_y_le, test_size=0.2, random_state=42)
# 2. le-norm feature - ori target
stdMath_x_le_norm = stdMath_df_le_norm.drop(columns=['G3'])
stdMath_y_le_norm = stdMath_df_le['G3']  # Target variable
stdMath_x_train_le_norm, stdMath_x_test_le_norm, stdMath_y_train_le_norm, stdMath_y_test_le_norm = train_test_split(stdMath_x_le_norm, stdMath_y_le_norm, test_size=0.2, random_state=42)
# 3. le-std feature - ori target
stdMath_x_le_std = stdMath_df_le_std.drop(columns=['G3'])
stdMath_y_le_std = stdMath_df_le['G3']  # Target variable
stdMath_x_train_le_std, stdMath_x_test_le_std, stdMath_y_train_le_std, stdMath_y_test_le_std = train_test_split(stdMath_x_le_std, stdMath_y_le_std, test_size=0.2, random_state=42)
# 4. ohe feature - ori target
stdMath_x_ohe = stdMath_df_ohe.drop(columns=['G3'])
stdMath_y_ohe = stdMath_df_ohe['G3']  # Target variable
stdMath_x_train_ohe, stdMath_x_test_ohe, stdMath_y_train_ohe, stdMath_y_test_ohe = train_test_split(stdMath_x_ohe, stdMath_y_ohe, test_size=0.2, random_state=42)
# 5. ohe-norm feature - ori target
stdMath_x_ohe_norm = stdMath_df_ohe_norm.drop(columns=['G3'])
stdMath_y_ohe_norm = stdMath_df_ohe['G3']  # Target variable
stdMath_x_train_ohe_norm, stdMath_x_test_ohe_norm, stdMath_y_train_ohe_norm, stdMath_y_test_ohe_norm = train_test_split(stdMath_x_ohe_norm, stdMath_y_ohe_norm, test_size=0.2, random_state=42)
# 6. ohe-std feature - ori target
stdMath_x_ohe_std = stdMath_df_ohe_std.drop(columns=['G3'])
stdMath_y_ohe_std = stdMath_df_ohe['G3']  # Target variable
stdMath_x_train_ohe_std, stdMath_x_test_ohe_std, stdMath_y_train_ohe_std, stdMath_y_test_ohe_std = train_test_split(stdMath_x_ohe_std, stdMath_y_ohe_std, test_size=0.2, random_state=42)
# 7. le feature - le target
stdMath_x_le = stdMath_df_le.drop(columns=['G3'])
stdMath_y_le = stdMath_df_le['G3']  # Target variable
stdMath_x_train_le, stdMath_x_test_le, stdMath_y_train_le, stdMath_y_test_le = train_test_split(stdMath_x_le, stdMath_y_le, test_size=0.2, random_state=42)
# 8. le-norm feature - le target
stdMath_x_le_norm = stdMath_df_le_norm.drop(columns=['G3'])
stdMath_y_le_norm = stdMath_df_le['G3']  # Target variable
stdMath_x_train_le_norm, stdMath_x_test_le_norm, stdMath_y_train_le_norm, stdMath_y_test_le_norm = train_test_split(stdMath_x_le_norm, stdMath_y_le_norm, test_size=0.2, random_state=42)
# 9. le-std feature - le target
stdMath_x_le_std = stdMath_df_le_std.drop(columns=['G3'])
stdMath_y_le_std = stdMath_df_le['G3']  # Target variable
stdMath_x_train_le_std, stdMath_x_test_le_std, stdMath_y_train_le_std, stdMath_y_test_le_std = train_test_split(stdMath_x_le_std, stdMath_y_le_std, test_size=0.2, random_state=42)
# 10. ohe feature - ohe target
stdMath_x_ohe = stdMath_df_ohe.drop(columns=['G3'])
stdMath_y_ohe = stdMath_df_ohe['G3']  # Target variable
stdMath_x_train_ohe, stdMath_x_test_ohe, stdMath_y_train_ohe, stdMath_y_test_ohe = train_test_split(stdMath_x_ohe, stdMath_y_ohe, test_size=0.2, random_state=42)
# 11. ohe-norm feature - ohe target
stdMath_x_ohe_norm = stdMath_df_ohe_norm.drop(columns=['G3'])
stdMath_y_ohe_norm = stdMath_df_ohe['G3']  # Target variable
stdMath_x_train_ohe_norm, stdMath_x_test_ohe_norm, stdMath_y_train_ohe_norm, stdMath_y_test_ohe_norm = train_test_split(stdMath_x_ohe_norm, stdMath_y_ohe_norm, test_size=0.2, random_state=42)
# 12. ohe-std feature - ohe target
stdMath_x_ohe_std = stdMath_df_ohe_std.drop(columns=['G3'])
stdMath_y_ohe_std = stdMath_df_ohe['G3']  # Target variable
stdMath_x_train_ohe_std, stdMath_x_test_ohe_std, stdMath_y_train_ohe_std, stdMath_y_test_ohe_std = train_test_split(stdMath_x_ohe_std, stdMath_y_ohe_std, test_size=0.2, random_state=42)

# Student Portuguese dataset
# 1. le feature - ori target
stdPor_x_le = stdPor_df_le.drop(columns=['G3'])
stdPor_y_le = stdPor_df_le['G3']  # Target variable
stdPor_x_train_le, stdPor_x_test_le, stdPor_y_train_le, stdPor_y_test_le = train_test_split(stdPor_x_le, stdPor_y_le, test_size=0.2, random_state=42)
# 2. le-norm feature - ori target
stdPor_x_le_norm = stdPor_df_le_norm.drop(columns=['G3'])
stdPor_y_le_norm = stdPor_df_le['G3']  # Target variable
stdPor_x_train_le_norm, stdPor_x_test_le_norm, stdPor_y_train_le_norm, stdPor_y_test_le_norm = train_test_split(stdPor_x_le_norm, stdPor_y_le_norm, test_size=0.2, random_state=42)
# 3. le-std feature - ori target
stdPor_x_le_std = stdPor_df_le_std.drop(columns=['G3'])
stdPor_y_le_std = stdPor_df_le['G3']  # Target variable
stdPor_x_train_le_std, stdPor_x_test_le_std, stdPor_y_train_le_std, stdPor_y_test_le_std = train_test_split(stdPor_x_le_std, stdPor_y_le_std, test_size=0.2, random_state=42)
# 4. ohe feature - ori target
stdPor_x_ohe = stdPor_df_ohe.drop(columns=['G3'])
stdPor_y_ohe = stdPor_df_ohe['G3']  # Target variable
stdPor_x_train_ohe, stdPor_x_test_ohe, stdPor_y_train_ohe, stdPor_y_test_ohe = train_test_split(stdPor_x_ohe, stdPor_y_ohe, test_size=0.2, random_state=42)
# 5. ohe-norm feature - ori target
stdPor_x_ohe_norm = stdPor_df_ohe_norm.drop(columns=['G3'])
stdPor_y_ohe_norm = stdPor_df_ohe['G3']  # Target variable
stdPor_x_train_ohe_norm, stdPor_x_test_ohe_norm, stdPor_y_train_ohe_norm, stdPor_y_test_ohe_norm = train_test_split(stdPor_x_ohe_norm, stdPor_y_ohe_norm, test_size=0.2, random_state=42)
# 6. ohe-std feature - ori target
stdPor_x_ohe_std = stdPor_df_ohe_std.drop(columns=['G3'])
stdPor_y_ohe_std = stdPor_df_ohe['G3']  # Target variable
stdPor_x_train_ohe_std, stdPor_x_test_ohe_std, stdPor_y_train_ohe_std, stdPor_y_test_ohe_std = train_test_split(stdPor_x_ohe_std, stdPor_y_ohe_std, test_size=0.2, random_state=42)
# 7. le feature - le target
stdPor_x_le = stdPor_df_le.drop(columns=['G3'])
stdPor_y_le = stdPor_df_le['G3']  # Target variable
stdPor_x_train_le, stdPor_x_test_le, stdPor_y_train_le, stdPor_y_test_le = train_test_split(stdPor_x_le, stdPor_y_le, test_size=0.2, random_state=42)
# 8. le-norm feature - le target
stdPor_x_le_norm = stdPor_df_le_norm.drop(columns=['G3'])
stdPor_y_le_norm = stdPor_df_le['G3']  # Target variable
stdPor_x_train_le_norm, stdPor_x_test_le_norm, stdPor_y_train_le_norm, stdPor_y_test_le_norm = train_test_split(stdPor_x_le_norm, stdPor_y_le_norm, test_size=0.2, random_state=42)
# 9. le-std feature - le target
stdPor_x_le_std = stdPor_df_le_std.drop(columns=['G3'])
stdPor_y_le_std = stdPor_df_le['G3']  # Target variable
stdPor_x_train_le_std, stdPor_x_test_le_std, stdPor_y_train_le_std, stdPor_y_test_le_std = train_test_split(stdPor_x_le_std, stdPor_y_le_std, test_size=0.2, random_state=42)
# 10. ohe feature - ohe target
stdPor_x_ohe = stdPor_df_ohe.drop(columns=['G3'])
stdPor_y_ohe = stdPor_df_ohe['G3']  # Target variable
stdPor_x_train_ohe, stdPor_x_test_ohe, stdPor_y_train_ohe, stdPor_y_test_ohe = train_test_split(stdPor_x_ohe, stdPor_y_ohe, test_size=0.2, random_state=42)
# 11. ohe-norm feature - ohe target
stdPor_x_ohe_norm = stdPor_df_ohe_norm.drop(columns=['G3'])
stdPor_y_ohe_norm = stdPor_df_ohe['G3']  # Target variable
stdPor_x_train_ohe_norm, stdPor_x_test_ohe_norm, stdPor_y_train_ohe_norm, stdPor_y_test_ohe_norm = train_test_split(stdPor_x_ohe_norm, stdPor_y_ohe_norm, test_size=0.2, random_state=42)
# 12. ohe-std feature - ohe target
stdPor_x_ohe_std = stdPor_df_ohe_std.drop(columns=['G3'])
stdPor_y_ohe_std = stdPor_df_ohe['G3']  # Target variable
stdPor_x_train_ohe_std, stdPor_x_test_ohe_std, stdPor_y_train_ohe_std, stdPor_y_test_ohe_std = train_test_split(stdPor_x_ohe_std, stdPor_y_ohe_std, test_size=0.2, random_state=42)

In [ ]:
# Step 4: Build , Train, Evaluate the model
# * Both dataset target are continuous, use MLPRegressor
# * Evaluate the model with error based metrics (MAE, MSE, RMSE, R2)
# * Plot the training history
# * Plot the prediction vs actual target

result = pd.DataFrame(columns=["Mean Absolute Error", "Mean Squared Error", "Root Mean Squared Error", "R2 Score"])


# A function to train and evaluate the model
def train_and_evaluate(model, x_train, y_train, x_test, y_test, tittle):
    # * Train the model
    model.fit(x_train, y_train)

    # * Evaluate the model with error based metrics (MAE, MSE, RMSE)
    y_pred = model.predict(x_test)  # Get predicted target
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    result.loc[tittle] = [mae, mse, rmse, r2]

    # * Plot the training history and prediction vs actual target
    # Plot the training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(model.loss_curve_)
    plt.title('Training History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    # Plot the prediction vs actual target
    plt.subplot(1, 2, 2)
    plt.scatter(y_test, y_pred)
    plt.title('Prediction vs Actual Target')
    plt.xlabel('Actual Target')
    plt.ylabel('Predicted Target')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.suptitle(tittle, fontsize=20)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    # plt.savefig("output/1. " + tittle + '.png', dpi=600)
    plt.close()


# Step 4.1 Build the model
# For wine dataset
# 11 features with 4898 samples
wineModel = MLPRegressor(
    hidden_layer_sizes=(16, 8),
    activation='relu',
    solver='adam',
    learning_rate='adaptive',
    learning_rate_init=0.025,
    max_iter=1000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.2,
    # verbose=True,
)

# For student dataset
# 30 features with 649 samples
stdModel = MLPRegressor(
    hidden_layer_sizes=(32, 16, 8),
    activation='relu',
    solver='adam',
    learning_rate='adaptive',
    learning_rate_init=0.025,
    max_iter=1000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.2,
    # verbose=True,
)

# Standard MLPRegressor Model
standardModel = MLPRegressor(
    hidden_layer_sizes=(16, 8, 8),
    activation='relu',
    solver='adam',
    learning_rate='adaptive',
    learning_rate_init=0.025,
    max_iter=1000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.2,
    # verbose=True,
)

In [6]:
# Step 4.2 Train and Evaluate the model with custom model
# For Red Wine Dataset
# ! Possible Combination of feature and target for wine dataset
# ! 1. ori feature - ori target
# ! 2. norm feature - norm target
# ! 3. std feature - std target
# ! 4. norm feature - ori target
# ! 5. std feature - ori target
# 1. ori feature - ori target
train_and_evaluate(wineModel, wineRed_x_train_ori, wineRed_y_train_ori, wineRed_x_test_ori, wineRed_y_test_ori, "Wine Red [wineModel] - [ori feature - ori target]")
# 2. norm feature - norm target
train_and_evaluate(wineModel, wineRed_x_train_norm, wineRed_y_train_norm, wineRed_x_test_norm, wineRed_y_test_norm, "Wine Red [wineModel] - [norm feature - norm target]")
# 3. std feature - std target
train_and_evaluate(wineModel, wineRed_x_train_std, wineRed_y_train_std, wineRed_x_test_std, wineRed_y_test_std, "Wine Red [wineModel] - [std feature - std target]")
# 4. norm feature - ori target
train_and_evaluate(wineModel, wineRed_x_train_norm_ori, wineRed_y_train_norm_ori, wineRed_x_test_norm_ori, wineRed_y_test_norm_ori, "Wine Red [wineModel] - [norm feature - ori target]")
# 5. std feature - ori target
train_and_evaluate(wineModel, wineRed_x_train_std_ori, wineRed_y_train_std_ori, wineRed_x_test_std_ori, wineRed_y_test_std_ori, "Wine Red [wineModel] - [std feature - ori target]")

# For White Wine Dataset
# 1. ori feature - ori target
train_and_evaluate(wineModel, wineWhite_x_train_ori, wineWhite_y_train_ori, wineWhite_x_test_ori, wineWhite_y_test_ori, "Wine White [wineModel] - [ori feature - ori target]")
# 2. norm feature - norm target
train_and_evaluate(wineModel, wineWhite_x_train_norm, wineWhite_y_train_norm, wineWhite_x_test_norm, wineWhite_y_test_norm, "Wine White [wineModel] - [norm feature - norm target]")
# 3. std feature - std target
train_and_evaluate(wineModel, wineWhite_x_train_std, wineWhite_y_train_std, wineWhite_x_test_std, wineWhite_y_test_std, "Wine White [wineModel] - [std feature - std target]")
# 4. norm feature - ori target
train_and_evaluate(wineModel, wineWhite_x_train_norm_ori, wineWhite_y_train_norm_ori, wineWhite_x_test_norm_ori, wineWhite_y_test_norm_ori, "Wine White [wineModel] - [norm feature - ori target]")
# 5. std feature - ori target
train_and_evaluate(wineModel, wineWhite_x_train_std_ori, wineWhite_y_train_std_ori, wineWhite_x_test_std_ori, wineWhite_y_test_std_ori, "Wine White [wineModel] - [std feature - ori target]")


# For Student Math Dataset
# ! Possible Combination of feature and target for student dataset
# ! 1. le feature - ori target
# ! 2. le-norm feature - ori target
# ! 3. le-std feature - ori target
# ! 4. ohe feature - ori target
# ! 5. ohe-norm feature - ori target
# ! 6. ohe-std feature - ori target
# ! 7. le feature - le target
# ! 8. le-norm feature - le target
# ! 9. le-std feature - le target
# ! 10. ohe feature - ohe target
# ! 11. ohe-norm feature - ohe target
# ! 12. ohe-std feature - ohe target
# 1. le feature - ori target
train_and_evaluate(stdModel, stdMath_x_train_le, stdMath_y_train_le, stdMath_x_test_le, stdMath_y_test_le, "Student Math [stdModel] - [le feature - ori target]")
# 2. le-norm feature - ori target
train_and_evaluate(stdModel, stdMath_x_train_le_norm, stdMath_y_train_le_norm, stdMath_x_test_le_norm, stdMath_y_test_le_norm, "Student Math [stdModel] - [le-norm feature - ori target]")
# 3. le-std feature - ori target
train_and_evaluate(stdModel, stdMath_x_train_le_std, stdMath_y_train_le_std, stdMath_x_test_le_std, stdMath_y_test_le_std, "Student Math [stdModel] - [le-std feature - ori target]")
# 4. ohe feature - ori target
train_and_evaluate(stdModel, stdMath_x_train_ohe, stdMath_y_train_ohe, stdMath_x_test_ohe, stdMath_y_test_ohe, "Student Math [stdModel] - [ohe feature - ori target]")
# 5. ohe-norm feature - ori target
train_and_evaluate(stdModel, stdMath_x_train_ohe_norm, stdMath_y_train_ohe_norm, stdMath_x_test_ohe_norm, stdMath_y_test_ohe_norm, "Student Math [stdModel] - [ohe-norm feature - ori target]")
# 6. ohe-std feature - ori target
train_and_evaluate(stdModel, stdMath_x_train_ohe_std, stdMath_y_train_ohe_std, stdMath_x_test_ohe_std, stdMath_y_test_ohe_std, "Student Math [stdModel] - [ohe-std feature - ori target]")
# 7. le feature - le target
train_and_evaluate(stdModel, stdMath_x_train_le, stdMath_y_train_le, stdMath_x_test_le, stdMath_y_test_le, "Student Math [stdModel] - [le feature - le target]")
# 8. le-norm feature - le target
train_and_evaluate(stdModel, stdMath_x_train_le_norm, stdMath_y_train_le_norm, stdMath_x_test_le_norm, stdMath_y_test_le_norm, "Student Math [stdModel] - [le-norm feature - le target]")
# 9. le-std feature - le target
train_and_evaluate(stdModel, stdMath_x_train_le_std, stdMath_y_train_le_std, stdMath_x_test_le_std, stdMath_y_test_le_std, "Student Math [stdModel] - [le-std feature - le target]")
# 10. ohe feature - ohe target
train_and_evaluate(stdModel, stdMath_x_train_ohe, stdMath_y_train_ohe, stdMath_x_test_ohe, stdMath_y_test_ohe, "Student Math [stdModel] - [ohe feature - ohe target]")
# 11. ohe-norm feature - ohe target
train_and_evaluate(stdModel, stdMath_x_train_ohe_norm, stdMath_y_train_ohe_norm, stdMath_x_test_ohe_norm, stdMath_y_test_ohe_norm, "Student Math [stdModel] - [ohe-norm feature - ohe target]")
# 12. ohe-std feature - ohe target
train_and_evaluate(stdModel, stdMath_x_train_ohe_std, stdMath_y_train_ohe_std, stdMath_x_test_ohe_std, stdMath_y_test_ohe_std, "Student Math [stdModel] - [ohe-std feature - ohe target]")

# For Student Portuguese Dataset
# 1. le feature - ori target
train_and_evaluate(stdModel, stdPor_x_train_le, stdPor_y_train_le, stdPor_x_test_le, stdPor_y_test_le, "Student Portuguese [stdModel] - [le feature - ori target]")
# 2. le-norm feature - ori target
train_and_evaluate(stdModel, stdPor_x_train_le_norm, stdPor_y_train_le_norm, stdPor_x_test_le_norm, stdPor_y_test_le_norm, "Student Portuguese [stdModel] - [le-norm feature - ori target]")
# 3. le-std feature - ori target
train_and_evaluate(stdModel, stdPor_x_train_le_std, stdPor_y_train_le_std, stdPor_x_test_le_std, stdPor_y_test_le_std, "Student Portuguese [stdModel] - [le-std feature - ori target]")
# 4. ohe feature - ori target
train_and_evaluate(stdModel, stdPor_x_train_ohe, stdPor_y_train_ohe, stdPor_x_test_ohe, stdPor_y_test_ohe, "Student Portuguese [stdModel] - [ohe feature - ori target]")
# 5. ohe-norm feature - ori target
train_and_evaluate(stdModel, stdPor_x_train_ohe_norm, stdPor_y_train_ohe_norm, stdPor_x_test_ohe_norm, stdPor_y_test_ohe_norm, "Student Portuguese [stdModel] - [ohe-norm feature - ori target]")
# 6. ohe-std feature - ori target
train_and_evaluate(stdModel, stdPor_x_train_ohe_std, stdPor_y_train_ohe_std, stdPor_x_test_ohe_std, stdPor_y_test_ohe_std, "Student Portuguese [stdModel] - [ohe-std feature - ori target]")
# 7. le feature - le target
train_and_evaluate(stdModel, stdPor_x_train_le, stdPor_y_train_le, stdPor_x_test_le, stdPor_y_test_le, "Student Portuguese [stdModel] - [le feature - le target]")
# 8. le-norm feature - le target
train_and_evaluate(stdModel, stdPor_x_train_le_norm, stdPor_y_train_le_norm, stdPor_x_test_le_norm, stdPor_y_test_le_norm, "Student Portuguese [stdModel] - [le-norm feature - le target]")
# 9. le-std feature - le target
train_and_evaluate(stdModel, stdPor_x_train_le_std, stdPor_y_train_le_std, stdPor_x_test_le_std, stdPor_y_test_le_std, "Student Portuguese [stdModel] - [le-std feature - le target]")
# 10. ohe feature - ohe target
train_and_evaluate(stdModel, stdPor_x_train_ohe, stdPor_y_train_ohe, stdPor_x_test_ohe, stdPor_y_test_ohe, "Student Portuguese [stdModel] - [ohe feature - ohe target]")
# 11. ohe-norm feature - ohe target
train_and_evaluate(stdModel, stdPor_x_train_ohe_norm, stdPor_y_train_ohe_norm, stdPor_x_test_ohe_norm, stdPor_y_test_ohe_norm, "Student Portuguese [stdModel] - [ohe-norm feature - ohe target]")
# 12. ohe-std feature - ohe target
train_and_evaluate(stdModel, stdPor_x_train_ohe_std, stdPor_y_train_ohe_std, stdPor_x_test_ohe_std, stdPor_y_test_ohe_std, "Student Portuguese [stdModel] - [ohe-std feature - ohe target]")

In [7]:
# Step 4.2 Train and Evaluate the model with custom model
# For Red Wine Dataset
# ! Possible Combination of feature and target for wine dataset
# ! 1. ori feature - ori target
# ! 2. norm feature - norm target
# ! 3. std feature - std target
# ! 4. norm feature - ori target
# ! 5. std feature - ori target
# 1. ori feature - ori target
train_and_evaluate(standardModel, wineRed_x_train_ori, wineRed_y_train_ori, wineRed_x_test_ori, wineRed_y_test_ori, "Wine Red [standardModel] - [ori feature - ori target]")
# 2. norm feature - norm target
train_and_evaluate(standardModel, wineRed_x_train_norm, wineRed_y_train_norm, wineRed_x_test_norm, wineRed_y_test_norm, "Wine Red [standardModel] - [norm feature - norm target]")
# 3. std feature - std target
train_and_evaluate(standardModel, wineRed_x_train_std, wineRed_y_train_std, wineRed_x_test_std, wineRed_y_test_std, "Wine Red [standardModel] - [std feature - std target]")
# 4. norm feature - ori target
train_and_evaluate(standardModel, wineRed_x_train_norm_ori, wineRed_y_train_norm_ori, wineRed_x_test_norm_ori, wineRed_y_test_norm_ori, "Wine Red [standardModel] - [norm feature - ori target]")
# 5. std feature - ori target
train_and_evaluate(standardModel, wineRed_x_train_std_ori, wineRed_y_train_std_ori, wineRed_x_test_std_ori, wineRed_y_test_std_ori, "Wine Red [standardModel] - [std feature - ori target]")

# For White Wine Dataset
# 1. ori feature - ori target
train_and_evaluate(standardModel, wineWhite_x_train_ori, wineWhite_y_train_ori, wineWhite_x_test_ori, wineWhite_y_test_ori, "Wine White [standardModel] - [ori feature - ori target]")
# 2. norm feature - norm target
train_and_evaluate(standardModel, wineWhite_x_train_norm, wineWhite_y_train_norm, wineWhite_x_test_norm, wineWhite_y_test_norm, "Wine White [standardModel] - [norm feature - norm target]")
# 3. std feature - std target
train_and_evaluate(standardModel, wineWhite_x_train_std, wineWhite_y_train_std, wineWhite_x_test_std, wineWhite_y_test_std, "Wine White [standardModel] - [std feature - std target]")
# 4. norm feature - ori target
train_and_evaluate(standardModel, wineWhite_x_train_norm_ori, wineWhite_y_train_norm_ori, wineWhite_x_test_norm_ori, wineWhite_y_test_norm_ori, "Wine White [standardModel] - [norm feature - ori target]")
# 5. std feature - ori target
train_and_evaluate(standardModel, wineWhite_x_train_std_ori, wineWhite_y_train_std_ori, wineWhite_x_test_std_ori, wineWhite_y_test_std_ori, "Wine White [standardModel] - [std feature - ori target]")


# For Student Math Dataset
# ! Possible Combination of feature and target for student dataset
# ! 1. le feature - ori target
# ! 2. le-norm feature - ori target
# ! 3. le-std feature - ori target
# ! 4. ohe feature - ori target
# ! 5. ohe-norm feature - ori target
# ! 6. ohe-std feature - ori target
# ! 7. le feature - le target
# ! 8. le-norm feature - le target
# ! 9. le-std feature - le target
# ! 10. ohe feature - ohe target
# ! 11. ohe-norm feature - ohe target
# ! 12. ohe-std feature - ohe target
# 1. le feature - ori target
train_and_evaluate(standardModel, stdMath_x_train_le, stdMath_y_train_le, stdMath_x_test_le, stdMath_y_test_le, "Student Math [standardModel] - [le feature - ori target]")
# 2. le-norm feature - ori target
train_and_evaluate(standardModel, stdMath_x_train_le_norm, stdMath_y_train_le_norm, stdMath_x_test_le_norm, stdMath_y_test_le_norm, "Student Math [standardModel] - [le-norm feature - ori target]")
# 3. le-std feature - ori target
train_and_evaluate(standardModel, stdMath_x_train_le_std, stdMath_y_train_le_std, stdMath_x_test_le_std, stdMath_y_test_le_std, "Student Math [standardModel] - [le-std feature - ori target]")
# 4. ohe feature - ori target
train_and_evaluate(standardModel, stdMath_x_train_ohe, stdMath_y_train_ohe, stdMath_x_test_ohe, stdMath_y_test_ohe, "Student Math [standardModel] - [ohe feature - ori target]")
# 5. ohe-norm feature - ori target
train_and_evaluate(standardModel, stdMath_x_train_ohe_norm, stdMath_y_train_ohe_norm, stdMath_x_test_ohe_norm, stdMath_y_test_ohe_norm, "Student Math [standardModel] - [ohe-norm feature - ori target]")
# 6. ohe-std feature - ori target
train_and_evaluate(standardModel, stdMath_x_train_ohe_std, stdMath_y_train_ohe_std, stdMath_x_test_ohe_std, stdMath_y_test_ohe_std, "Student Math [standardModel] - [ohe-std feature - ori target]")
# 7. le feature - le target
train_and_evaluate(standardModel, stdMath_x_train_le, stdMath_y_train_le, stdMath_x_test_le, stdMath_y_test_le, "Student Math [standardModel] - [le feature - le target]")
# 8. le-norm feature - le target
train_and_evaluate(standardModel, stdMath_x_train_le_norm, stdMath_y_train_le_norm, stdMath_x_test_le_norm, stdMath_y_test_le_norm, "Student Math [standardModel] - [le-norm feature - le target]")
# 9. le-std feature - le target
train_and_evaluate(standardModel, stdMath_x_train_le_std, stdMath_y_train_le_std, stdMath_x_test_le_std, stdMath_y_test_le_std, "Student Math [standardModel] - [le-std feature - le target]")
# 10. ohe feature - ohe target
train_and_evaluate(standardModel, stdMath_x_train_ohe, stdMath_y_train_ohe, stdMath_x_test_ohe, stdMath_y_test_ohe, "Student Math [standardModel] - [ohe feature - ohe target]")
# 11. ohe-norm feature - ohe target
train_and_evaluate(standardModel, stdMath_x_train_ohe_norm, stdMath_y_train_ohe_norm, stdMath_x_test_ohe_norm, stdMath_y_test_ohe_norm, "Student Math [standardModel] - [ohe-norm feature - ohe target]")
# 12. ohe-std feature - ohe target
train_and_evaluate(standardModel, stdMath_x_train_ohe_std, stdMath_y_train_ohe_std, stdMath_x_test_ohe_std, stdMath_y_test_ohe_std, "Student Math [standardModel] - [ohe-std feature - ohe target]")

# For Student Portuguese Dataset
# 1. le feature - ori target
train_and_evaluate(standardModel, stdPor_x_train_le, stdPor_y_train_le, stdPor_x_test_le, stdPor_y_test_le, "Student Portuguese [standardModel] - [le feature - ori target]")
# 2. le-norm feature - ori target
train_and_evaluate(standardModel, stdPor_x_train_le_norm, stdPor_y_train_le_norm, stdPor_x_test_le_norm, stdPor_y_test_le_norm, "Student Portuguese [standardModel] - [le-norm feature - ori target]")
# 3. le-std feature - ori target
train_and_evaluate(standardModel, stdPor_x_train_le_std, stdPor_y_train_le_std, stdPor_x_test_le_std, stdPor_y_test_le_std, "Student Portuguese [standardModel] - [le-std feature - ori target]")
# 4. ohe feature - ori target
train_and_evaluate(standardModel, stdPor_x_train_ohe, stdPor_y_train_ohe, stdPor_x_test_ohe, stdPor_y_test_ohe, "Student Portuguese [standardModel] - [ohe feature - ori target]")
# 5. ohe-norm feature - ori target
train_and_evaluate(standardModel, stdPor_x_train_ohe_norm, stdPor_y_train_ohe_norm, stdPor_x_test_ohe_norm, stdPor_y_test_ohe_norm, "Student Portuguese [standardModel] - [ohe-norm feature - ori target]")
# 6. ohe-std feature - ori target
train_and_evaluate(standardModel, stdPor_x_train_ohe_std, stdPor_y_train_ohe_std, stdPor_x_test_ohe_std, stdPor_y_test_ohe_std, "Student Portuguese [standardModel] - [ohe-std feature - ori target]")
# 7. le feature - le target
train_and_evaluate(standardModel, stdPor_x_train_le, stdPor_y_train_le, stdPor_x_test_le, stdPor_y_test_le, "Student Portuguese [standardModel] - [le feature - le target]")
# 8. le-norm feature - le target
train_and_evaluate(standardModel, stdPor_x_train_le_norm, stdPor_y_train_le_norm, stdPor_x_test_le_norm, stdPor_y_test_le_norm, "Student Portuguese [standardModel] - [le-norm feature - le target]")
# 9. le-std feature - le target
train_and_evaluate(standardModel, stdPor_x_train_le_std, stdPor_y_train_le_std, stdPor_x_test_le_std, stdPor_y_test_le_std, "Student Portuguese [standardModel] - [le-std feature - le target]")
# 10. ohe feature - ohe target
train_and_evaluate(standardModel, stdPor_x_train_ohe, stdPor_y_train_ohe, stdPor_x_test_ohe, stdPor_y_test_ohe, "Student Portuguese [standardModel] - [ohe feature - ohe target]")
# 11. ohe-norm feature - ohe target
train_and_evaluate(standardModel, stdPor_x_train_ohe_norm, stdPor_y_train_ohe_norm, stdPor_x_test_ohe_norm, stdPor_y_test_ohe_norm, "Student Portuguese [standardModel] - [ohe-norm feature - ohe target]")
# 12. ohe-std feature - ohe target
train_and_evaluate(standardModel, stdPor_x_train_ohe_std, stdPor_y_train_ohe_std, stdPor_x_test_ohe_std, stdPor_y_test_ohe_std, "Student Portuguese [standardModel] - [ohe-std feature - ohe target]")

In [11]:
# Step 5: Compare the result
# Compare the result
pd.set_option("display.max_rows", None)
display(result)

# Step 6: Save the result
result.to_csv("output/2. Evaluation Result.csv")

,Mean Absolute Error,Mean Squared Error,Root Mean Squared Error,R2 Score
Wine Red [wineModel] - [ori feature - ori target],0.509317,0.403448,0.635176,0.382640
Wine Red [wineModel] - [norm feature - norm target],0.101259,0.015835,0.125837,0.394228
Wine Red [wineModel] - [std feature - std target],0.605702,0.566673,0.752777,0.434841
Wine Red [wineModel] - [norm feature - ori target],0.506266,0.404332,0.635871,0.381287
Wine Red [wineModel] - [std feature - ori target],0.504871,0.403792,0.635447,0.382113
Wine White [wineModel] - [ori feature - ori target],0.594063,0.580772,0.762084,0.250107
Wine White [wineModel] - [norm feature - norm target],0.099010,0.015981,0.126415,0.257167
Wine White [wineModel] - [std feature - std target],0.617380,0.632820,0.795500,0.359236
Wine White [wineModel] - [norm feature - ori target],0.596024,0.590501,0.768440,0.237545
Wine White [wineModel] - [std feature - ori target],0.553870,0.504916,0.710574,0.348052
